In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import json
import os
import re
import pandas as pd
import numpy as np

import drive_access
import parsemadzu

re_lcd = re.compile(r"(.*)\.lcd$")
re_txt = re.compile(r"(.*)\.txt$")

files = drive_access.list_files()
print(len(files))

986


In [16]:
def get_path(file: dict) -> str:
    """Find path to file, i.e. its folder location."""
    current_path = ""
    current_file = file
    for i in range(5):
        parents = current_file.get("parents", [])
        parent = parents[0] if parents else None
        
        if parent is None:
            break
        
        parent_search = [f for f in files if f["id"] == parent]
        
        if parent_search is None or len(parent_search) == 0:
            break
        
        parent_file = parent_search[0]
        current_path = os.path.join(parent_file["name"], current_path)
        current_file = parent_file
        
    return current_path

get_path(files[3])

'Project1/2021/10_October/20211006/'

In [17]:
#[f for f in files if "1-LVxSt2qo-5ioi3er3t3QsatEPo9V3-C" in f.get("parents", [])]

rows = []
columns = ["name", "drive_id", "path", "txt_drive_id"]

lcd_files = [f for f in files if re_lcd.match(f["name"])]

for lcd_file in lcd_files:
    txt_file_name = os.path.splitext(lcd_file["name"])[0] + ".txt"
    
    # Find files in same folder with same name but .txt extension instead of .lcd
    txt_file_search = [f for f in files if f["name"] == txt_file_name and f["parents"] == lcd_file["parents"]]

    txt_file = None
    if txt_file_search:
        txt_file = txt_file_search[0]
        
    rows.append([
        lcd_file["name"],
        lcd_file["id"],
        get_path(lcd_file),
        txt_file["id"] if txt_file else np.nan
    ])
    
df = pd.DataFrame(rows, columns=columns).sort_values("name")
df_has_txt = df[np.logical_not(pd.isna(df["txt_drive_id"]))]
df_has_txt

,name,drive_id,path,txt_drive_id
354,20210802 NB_02082021_001.lcd,1OkwCIftgGRb5PbA_jWoNBNP2jGO4KpgJ,Project1/2021/08_August/20210802/,1vOKSQdvsSpySnJJC-YcvVavvm-ds0AZm
353,20210802 NB_02082021_002.lcd,1OOMswDfN5NuS_SV8UD3sAegK1Ki6LZF4,Project1/2021/08_August/20210802/,1vQoA0hNLg35YHeLcWd1o87xsj5IqfRR8
352,20210802 NB_02082021_003.lcd,1OKcnfNtHSnrxbAxPo_ojf4dATNfTrujM,Project1/2021/08_August/20210802/,1vO87O__PxPEDS8jW-UwotnEZK32gblMS
351,20210802 NB_02082021_004.lcd,1OO5cRK34Od8KZkucncBNXznEnnIY0vm2,Project1/2021/08_August/20210802/,1v5bmNTNP4wB8jlv9IQgruE4INte9t2Z9
350,20210802 NB_03082021_001.lcd,1Oo4cOShKICc29sdSkmI1LStgjyCt1ZlG,Project1/2021/08_August/20210802/,1vDM6-4aAd8OkAcLlndQEPM44Ij6Ukmjf
...,...,...,...,...
15,20211004_JB_04102021_033.lcd,1rxE8uZGgTe1qsVk_MnA1af_UfIGRNhjd,Project1/2021/10_October/20211004/,1uqr3GXKYpo3fVk-Gt3ObRPA3bxspfJst
10,20211004_JB_04102021_034.lcd,1s0fmNSKLkErX6UwW7fDFAFrVeyV3L9CB,Project1/2021/10_October/20211004/,1v-xcjUzKpzXIXr6a8QKlhTwSiGIALguh
14,20211004_JB_04102021_035.lcd,1s9CbYv0-OvpazSLHc9q1e0us4Cgu68ER,Project1/2021/10_October/20211004/,1ux53g-gkS4MKVq1v-LGIC5rpLsoI2QAn
9,20211004_JB_04102021_036.lcd,1sBMgor4eT7Z8ScqsgR3OzBbnHT-cLsMI,Project1/2021/10_October/20211004/,1v2hCeJf7GyIdhuo-HWcsQ8KIaA-GdF36


In [18]:
def combine_compound_concentrations(df_txt_files: pd.DataFrame) -> pd.DataFrame:
    """Combine a list of result files into a table of concentrations of compounds detected in each experiment."""
    
    rows = []

    for _, row_in in df_txt_files.iterrows():
        f = drive_access.get_file(row_in["txt_drive_id"])
        
        sections = parsemadzu.parse_sections(f)
        compounds = parsemadzu.get_compound_table(sections, detector="B")
        sample_info = parsemadzu.get_sample_information(sections)
        
        row_out = {
            "Acquired Time": sample_info["Acquired"],
            "Dilution Factor": sample_info["Dilution Factor"],
            "Sample Name": sample_info["Sample Name"],
            "Sample Type": sample_info["Sample Type"],
        }
            
        if compounds is None:
            print(f"Skipping {row_in['txt_drive_id']} because of empty compound list")
        else:
            compound_names = compounds["Name"]
            concentrations = compounds["Conc."]

            for compound_name, concentration in zip(compound_names.tolist(), concentrations.tolist()):
                row_out[compound_name] = concentration
        
        rows.append(row_out)

    df_concs = pd.DataFrame(rows)
    df_concs["txt_drive_id"] = df_txt_files["txt_drive_id"].to_numpy()
    
    return df_concs

df_compounds_combined = combine_compound_concentrations(df_has_txt)

Skipping 1wYvcddWSuiyl52AAp2wPQAGNlN1ls6Ix because of empty compound list
Skipping 1wNtEswlAclWHqo0wp_26QP9X85hWYaeC because of empty compound list
Skipping 1wOoZvqmWEvC1hOhFZXUFPpYL13yWUW8B because of empty compound list
Skipping 1wJRcl9Iloo7sBMyxJ6pozmGIH91bsD-k because of empty compound list
Skipping 1wGLyA_RWDro7_GC8Ehc2frKrqtCFo5ak because of empty compound list
Skipping 1w_d2dA-9Jr1tAcJCGBGaRMIcQRbTRI5x because of empty compound list
Skipping 1wKOTadlLtQXukmghyXI_kY9-bCPAZuu6 because of empty compound list
Skipping 1wXuMMcjKuEnqlFuSzy_w0sYPGtCpAeki because of empty compound list
Skipping 1wcyuTAPQCdKL86XrE3B42hjt9Wx7_-gx because of empty compound list
Skipping 1wrjslIh_cYdTM8mHEcSu8ELn3Q9ibF2j because of empty compound list
Skipping 1wnTpn6qi-19xv-5IqxmYZC3JNGDFquZg because of empty compound list
Skipping 1wtSyNNJJklBHWbz9ZoJ-wvqghMB5177T because of empty compound list
Skipping 1wk7rIN9sHEAMxSqzyxJJW8GZQIReAuGV because of empty compound list
Skipping 1xInLGNqZ4S_ydwqi674tB9GizzAr

In [19]:
df_upload = df_has_txt.merge(df_compounds_combined, on="txt_drive_id")
df_upload.head()

,name,drive_id,path,txt_drive_id,Acquired Time,Dilution Factor,Sample Name,Sample Type,Xylose,Glucose,Mannose,Arabinose,Galactose,Arabitol,Xylitol,Acetic Acid,Glycerol,Ethanol,Citric Acid
0,20210802 NB_02082021_001.lcd,1OkwCIftgGRb5PbA_jWoNBNP2jGO4KpgJ,Project1/2021/08_August/20210802/,1vOKSQdvsSpySnJJC-YcvVavvm-ds0AZm,02/08/2021 11:45:44,1,H2O001,2:Control,0.000,0.0,0.000,0.0,0.0,0.00,0.000,NaN,NaN,NaN,NaN
1,20210802 NB_02082021_002.lcd,1OOMswDfN5NuS_SV8UD3sAegK1Ki6LZF4,Project1/2021/08_August/20210802/,1vQoA0hNLg35YHeLcWd1o87xsj5IqfRR8,02/08/2021 12:05:19,1,H2O001,2:Control,0.000,0.0,0.232,0.0,0.0,0.00,0.000,NaN,NaN,NaN,NaN
2,20210802 NB_02082021_003.lcd,1OKcnfNtHSnrxbAxPo_ojf4dATNfTrujM,Project1/2021/08_August/20210802/,1vO87O__PxPEDS8jW-UwotnEZK32gblMS,02/08/2021 13:05:40,1,S08,0:Unknown,2.021,0.0,0.467,0.0,0.0,0.38,0.184,NaN,NaN,NaN,NaN
3,20210802 NB_02082021_004.lcd,1OO5cRK34Od8KZkucncBNXznEnnIY0vm2,Project1/2021/08_August/20210802/,1v5bmNTNP4wB8jlv9IQgruE4INte9t2Z9,02/08/2021 14:06:01,1,S01,0:Unknown,5.372,0.0,0.287,0.0,0.0,0.00,0.000,NaN,NaN,NaN,NaN
4,20210802 NB_03082021_001.lcd,1Oo4cOShKICc29sdSkmI1LStgjyCt1ZlG,Project1/2021/08_August/20210802/,1vDM6-4aAd8OkAcLlndQEPM44Ij6Ukmjf,03/08/2021 10:22:53,1,H2O,2:Control,0.000,0.0,0.245,0.0,0.0,0.00,0.000,NaN,NaN,NaN,NaN


## Save to google sheet

In [20]:
import gspread
import df2gspread.df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials


# https://docs.google.com/spreadsheets/d/14DSZKVv0xCJFgxJUDYk-hgTrLtFCkWhZLsexDM3G8L0/edit#gid=0
spreadsheet_id = "14DSZKVv0xCJFgxJUDYk-hgTrLtFCkWhZLsexDM3G8L0"

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
cred = ServiceAccountCredentials.from_json_keyfile_name(
            "secrets/biolabautomation-d5dc83b8ddf7.json", scope)
gc = gspread.authorize(cred)
d2g.upload(df_upload, spreadsheet_id, "autoimport_HPLC_runs", credentials=cred, row_names=False)

<Worksheet 'autoimport_HPLC_runs' id:0>